# EDA 

Notebook to explore and crear EDA to the cars detail data

In [10]:
import importlib
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from functions_data import * 
from definitions import *


#fd = importlib.reload(functions_data)

missing_columns_check = ['id','precio','km','año']

schema = {  'id':'string',
            'link':'string',
            'modelo':'category',
            'version':'string',
            'precio':'double',
            'ciudad':'category',
            'km':'double',
            'año':'int',
            'estado':'category',
            'ubicacion':'category',
            'placa':'category',
            'tipo_de_caja':'category',
            'combustible':'category',
            'cilindraje':'double',
            'color':'category',
            'puertas':'double',
            'airbags':'category',
            'direccion':'category',
}




# Define function to clean and preprocess DataFrame
def clean_dataframe(df:pd.DataFrame, schema:dict, missing_columns_check:list) -> pd.DataFrame:
    print(f'df original size: {df.shape} - columns: {df.columns}' )
    # Standarize column names
    df = standarize_column_names(df)
    # Remove duplicates
    df = df.drop_duplicates()
    # Handle missing values
    df = df.dropna(subset=missing_columns_check)

    # Format fields to match the schema
    for column, dtype in schema.items():
        if column in df.columns:
            if dtype == 'date' and df[column].dtype != 'datetime64[ns]':
                df[column] = pd.to_datetime(df[column])
            elif dtype == 'int' and df[column].dtype != 'int64':
                df[column] = df[column].astype('int')
            elif dtype == 'double' and df[column].dtype != 'float64':
                df[column] = df[column].str.replace('$','').str.replace('.','').str.replace(',','').str.replace(' km','').str.replace(' ','').astype('float')
            elif dtype == 'string' and df[column].dtype != 'object':
                df[column] = df[column].astype('str')
            elif dtype == 'category':
                df[column] = df[column].str.lower().str.replace('(', '').str.replace(')', '')\
                    .str.replace('.','').str.replace('/','_').str.replace('-','_').str.replace('__','_')\
                    .str.replace('á','a').str.replace('é','e').str.replace('í','i').str.replace('ó','o').str.replace('ú','u').astype('category')
    print(f'df cleaned size: {df.shape} - columns: {df.columns}' )
    return df

csv_file = f'data/car_details.csv'
df = pd.read_csv(f'../{csv_file}')
df = clean_dataframe(df, schema, missing_columns_check)
# create new variables
df['marca'] = df['modelo'].apply(lambda x : str(x).split(' ')[0])
df['placa'] = df['placa'].apply(lambda x : str(x).split(' ')[0])

#Elegir solo carros usados
df = df.query('estado == "usado"')

# Corregir algunas valores de las variables categoricas
df['color'] = df['color'].str.replace('morado','otro').str.replace('lila','otro').str.replace('no disponible','otro').str.replace('varios','otro')
df['combustible'] = df['combustible'].str.replace('sin especificar','gasolina')
df['tipo_de_caja'] = df['tipo_de_caja'].str.replace('0','mecanica').str.replace('secuencial','automatica')
df['airbags'] = df['airbags'].str.replace('2','airbags').str.replace('airbags','1')
df['direccion'] = df['direccion'].str.replace('electro_hidraulica','electrica').str.replace('asistencia','').str.replace(' ','').str.replace('asistida','hidraulica')
df['cilindraje'] = df['cilindraje'].apply(lambda x: round(x *10) * 100 if x >= 0 and x<10 else x)
df['modelo'] = df['modelo'].apply(lambda x: ' '.join(x.split(' ')[1:]) if len(x.split(' ')) > 1 else x)
df['modelo'] = df['modelo'].str.replace('benz','')
df['impar'] = df['placa'].apply(lambda x: int(x[-1]) % 2 if x[-1].isdigit() else -1)


df.head()
#df.groupby(by=['airbags']).count()['id'].sort_values(ascending=False)#.head(10)

df original size: (7187, 19) - columns: Index(['id', 'link', 'modelo', 'version', 'precio', 'ciudad', 'km', 'año',
       'ESTADO', 'UBICACIÓN', 'PLACA', 'TIPO DE CAJA', 'COMBUSTIBLE',
       'CILINDRAJE', 'COLOR', 'PUERTAS', 'AIRBAGS', 'DIRECCIÓN',
       'Unnamed: 18'],
      dtype='object')
df cleaned size: (6928, 18) - columns: Index(['id', 'link', 'modelo', 'version', 'precio', 'ciudad', 'km', 'año',
       'estado', 'ubicacion', 'placa', 'tipo_de_caja', 'combustible',
       'cilindraje', 'color', 'puertas', 'airbags', 'direccion'],
      dtype='object')


,id,link,modelo,version,precio,ciudad,km,año,estado,ubicacion,placa,tipo_de_caja,combustible,cilindraje,color,puertas,airbags,direccion,marca,impar
0,8458b8fb-2f95-42b1-a5bc-d454803e5c12,https://www.carroya.com/detalle/usado/bmw/x2/2...,x2,sDrive 20i Premium,110000000.0,cajica,67000.0,2019,usado,cajica,**3,automatica,gasolina,2000.0,negro,4.0,NaN,NaN,bmw,1
1,f502baa1-3537-448b-b391-80829644ff06,https://www.carroya.com/detalle/usado/volkswag...,jetta,NaN,53000000.0,bogota,115000.0,2019,usado,bogota,**0,automatica,gasolina,1400.0,gris,NaN,NaN,NaN,volkswagen,0
2,f674328c-d998-4a29-8bb0-8da5e00158bb,https://www.carroya.com/detalle/usado/nissan/q...,qashqai,2.0 Automatica Full Equipo,69500000.0,cali,66500.0,2016,usado,cali,**0,automatica,gasolina,2000.0,negro,NaN,NaN,NaN,nissan,0
3,c9a0bf4d-09d6-4544-a705-28f50fe8107e,https://www.carroya.com/detalle/usado/renault/...,clio,1.4 Dynamique,17900000.0,cali,118000.0,2007,usado,cali,**4,mecanica,gasolina,0.0,plateado,NaN,NaN,NaN,renault,0
4,2281829,https://www.carroya.com/detalle/usado/chevrole...,onix,1.0 Turbo LT Sedan Automatico,59000000.0,bogota,39000.0,2021,usado,bogota,**3,automatica,gasolina,1000.0,plateado,NaN,1,NaN,chevrolet,1


In [11]:
# Analisis de calidad de los datos con profiling

profile = ProfileReport(df, title="Profiling Report")
profile.to_file("../quality/quality_report.html")

Summarize dataset:  81%|████████▏ | 22/27 [00:02<00:00, 13.11it/s, Calculate auto correlation]    c:\Users\Msastoqu\OneDrive - Caracol Televisión S.A\Master\TFM\project\env\lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[9 9 8 ... 9 9 8]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\Msastoqu\OneDrive - Caracol Televisión S.A\Master\TFM\project\env\lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include 